In [1]:
import src.miax_util as miax_util
import pandas as pd
import numpy as np

In [2]:
# Parámetros
INITIAL_CAPITAL = 250_000
COMMISSION_RATE = 0.0023
COMMISSION_MIN = 23


In [3]:

# ------------------------------------------------------------
# 1. CARGA DE DATOS
# ------------------------------------------------------------
parquet = pd.read_parquet("resources/assignment_parquet.parquet")
portfolio = pd.read_csv("resources/portfolio_selections.csv", parse_dates=['rebal_date'])
rebalancing_dates = pd.read_parquet("resources/rebalancing_dates.parquet")


print(f"Capital inicial: ${INITIAL_CAPITAL:,.0f}")
print(f"Fechas de rebalanceo: {rebalancing_dates['date'].nunique()}")
print(f"Shape portfolio: {portfolio.shape}")

Capital inicial: $250,000
Fechas de rebalanceo: 133
Shape portfolio: (2660, 8)


In [4]:
# ------------------------------------------------------------
# 2. LÓGICA DE UN REBALANCEO (primer mes como ejemplo)
# ------------------------------------------------------------

# Estado inicial
cash = INITIAL_CAPITAL
holdings = {}  # diccionario {ticker: num_acciones}

# Primera fecha de rebalanceo
rebal_date = rebalancing_dates['date'].iloc[0]
print(f"Primer rebalanceo: {rebal_date}")

# Cartera objetivo (los 20 seleccionados)
target = portfolio[portfolio['rebal_date'] == rebal_date]['symbol'].tolist()
print(f"Cartera objetivo: {target}")

# Precios de ese día
prices_today = parquet[parquet['date'] == rebal_date][['symbol', 'open', 'close']].set_index('symbol')
print(f"\nPrecios disponibles: {len(prices_today)} tickers")
print(prices_today.loc[prices_today.index.isin(target)].head())

Primer rebalanceo: 2015-01-30 00:00:00
Cartera objetivo: ['LUV', 'EW', 'EA', 'VRTX', 'MNST', 'AGN-201503', 'BBWI', 'KR', 'DAL', 'COV-201501', 'KMX', 'LOW', 'SIAL-201511', 'CFN-201503', 'TEG-201506', 'BRCM-201601', 'MAR', 'SPLS-201709', 'REGN', 'ISRG']

Precios disponibles: 772 tickers
                   open       close
symbol                             
AGN-201503   221.352249  219.212708
BBWI          46.717205   46.155460
BRCM-201601   40.523926   41.942265
CFN-201503    59.470001   59.299999
DAL           43.200146   41.431156


In [5]:
# ------------------------------------------------------------
# PRIMER REBALANCEO: SOLO COMPRAMOS
# ------------------------------------------------------------

total_commissions = 0

# Capital por posición: 5% del capital total
position_value = cash * 0.05  # 12,500$

for ticker in target:
    if ticker not in prices_today.index:
        print(f"⚠️ {ticker} sin precio, se omite")
        continue

    close_price = prices_today.loc[ticker, 'close']

    # Número de acciones que podemos comprar
    num_shares = position_value / close_price

    # Coste de la operación
    cost = num_shares * close_price
    commission = max(cost * COMMISSION_RATE, COMMISSION_MIN)

    # Actualizamos estado
    cash -= (cost + commission)
    holdings[ticker] = num_shares
    total_commissions += commission

    print(f"{ticker}: {num_shares:.2f} acciones @ {close_price:.2f} | comisión: {commission:.2f}$")

print(f"\nCash restante: ${cash:,.2f}")
print(f"Comisiones pagadas: ${total_commissions:,.2f}")
print(f"Posiciones abiertas: {len(holdings)}")

LUV: 313.29 acciones @ 39.90 | comisión: 28.75$
EW: 598.33 acciones @ 20.89 | comisión: 28.75$
EA: 234.35 acciones @ 53.34 | comisión: 28.75$
VRTX: 113.49 acciones @ 110.14 | comisión: 28.75$
MNST: 641.30 acciones @ 19.49 | comisión: 28.75$
AGN-201503: 57.02 acciones @ 219.21 | comisión: 28.75$
BBWI: 270.82 acciones @ 46.16 | comisión: 28.75$
KR: 447.72 acciones @ 27.92 | comisión: 28.75$
DAL: 301.71 acciones @ 41.43 | comisión: 28.75$
⚠️ COV-201501 sin precio, se omite
KMX: 201.29 acciones @ 62.10 | comisión: 28.75$
LOW: 225.28 acciones @ 55.49 | comisión: 28.75$
SIAL-201511: 91.35 acciones @ 136.84 | comisión: 28.75$
CFN-201503: 210.79 acciones @ 59.30 | comisión: 28.75$
TEG-201506: 156.95 acciones @ 79.64 | comisión: 28.75$
BRCM-201601: 298.03 acciones @ 41.94 | comisión: 28.75$
MAR: 186.90 acciones @ 66.88 | comisión: 28.75$
SPLS-201709: 824.08 acciones @ 15.17 | comisión: 28.75$
REGN: 30.17 acciones @ 414.36 | comisión: 28.75$
ISRG: 227.51 acciones @ 54.94 | comisión: 28.75$

Cash

In [6]:
# Test con el segundo rebalanceo
rebal_date_2 = rebalancing_dates['date'].iloc[1]
target_2 = portfolio[portfolio['rebal_date'] == rebal_date_2]['symbol'].tolist()

print(f"Segundo rebalanceo: {rebal_date_2}")
print(f"Cartera actual: {list(holdings.keys())}")
print(f"Cartera objetivo: {target_2}")
print(f"Cash actual: ${cash:,.2f}")

holdings, cash, commissions = miax_util.rebalance(
    rebal_date_2, target_2, holdings, cash, parquet,
    COMMISSION_RATE, COMMISSION_MIN
)

print(f"\nCash tras rebalanceo: ${cash:,.2f}")
print(f"Posiciones: {len(holdings)}")
print(f"Comisiones: ${commissions:,.2f}")

Segundo rebalanceo: 2015-02-27 00:00:00
Cartera actual: ['LUV', 'EW', 'EA', 'VRTX', 'MNST', 'AGN-201503', 'BBWI', 'KR', 'DAL', 'KMX', 'LOW', 'SIAL-201511', 'CFN-201503', 'TEG-201506', 'BRCM-201601', 'MAR', 'SPLS-201709', 'REGN', 'ISRG']
Cartera objetivo: ['LUV', 'EA', 'MNST', 'EW', 'AGN-201503', 'KR', 'REGN', 'KDP', 'WELL', 'DAL', 'BBWI', 'SIAL-201511', 'GGP-201808', 'LOW', 'ANDV-201809', 'CFN-201503', 'MAC', 'AIV', 'SHW', 'CELG-201911']
Cash actual: $11,953.75

Cash tras rebalanceo: $0.00
Posiciones: 20
Comisiones: $445.59


In [9]:
# ------------------------------------------------------------
# 4. BUCLE COMPLETO
# ------------------------------------------------------------
# Reseteamos estado inicial
cash = INITIAL_CAPITAL
holdings = {}
total_commissions = 0
holdings_history = {}  # {rebal_date: {ticker: num_acciones}}

# Primer rebalanceo (solo compras)
first_date = rebalancing_dates['date'].iloc[0]
first_target = portfolio[portfolio['rebal_date'] == first_date]['symbol'].tolist()
prices_first = (parquet[parquet['date'] == first_date]
                [['symbol', 'open', 'close']]
                .set_index('symbol'))

position_value = (cash / 20) / (1 + COMMISSION_RATE)

for ticker in first_target:
    if ticker not in prices_first.index:
        continue
    close_price = prices_first.loc[ticker, 'close']
    num_shares = position_value / close_price
    cost = num_shares * close_price
    commission = max(cost * COMMISSION_RATE, COMMISSION_MIN)
    cash -= (cost + commission)
    holdings[ticker] = num_shares
    total_commissions += commission

# Guardamos snapshot del primer rebalanceo
holdings_history[first_date] = {'holdings': holdings.copy(), 'cash': cash}

# Rebalanceos siguientes
for _, row in rebalancing_dates.iloc[1:].iterrows():
    rebal_date = row['date']
    target = portfolio[portfolio['rebal_date'] == rebal_date]['symbol'].tolist()

    holdings, cash, commissions = miax_util.rebalance(
        rebal_date, target, holdings, cash, parquet,
        COMMISSION_RATE, COMMISSION_MIN
    )
    total_commissions += commissions

    # Guardamos snapshot tras cada rebalanceo
    holdings_history[rebal_date] = {'holdings': holdings.copy(), 'cash': cash}

print(f"Comisiones totales: ${total_commissions:,.2f}")
print(f"Snapshots guardadas: {len(holdings_history)}")

⚠️ PLL-201508 sin precio en 2015-09-30 00:00:00, queda en liquidez
⚠️ CB-201601 sin precio en 2016-01-29 00:00:00, queda en liquidez
⚠️ CB-201601 sin precio en 2016-02-29 00:00:00, queda en liquidez
⚠️ SNDK-201605 sin precio en 2016-05-31 00:00:00, queda en liquidez
⚠️ SCG-201812 sin precio en 2019-01-31 00:00:00, queda en liquidez
⚠️ ABMD-202212 sin precio en 2023-01-31 00:00:00, queda en liquidez
Comisiones totales: $106,769.50
Snapshots guardadas: 133


In [10]:
# ------------------------------------------------------------
# 5. VALOR DE LA CARTERA DÍA A DÍA
# ------------------------------------------------------------

# Días hábiles desde el primer rebalanceo hasta el final
trading_days = (parquet[parquet['date'] >= first_date][['date']]
                .drop_duplicates()
                .sort_values('date'))

portfolio_values = []

# Para cada día hábil, usamos la snapshot del último rebalanceo
rebal_dates_sorted = sorted(holdings_history.keys())

for day in trading_days['date']:
    # Encontramos el último rebalanceo anterior o igual a este día
    active_rebal = None
    for rd in rebal_dates_sorted:
        if rd <= day:
            active_rebal = rd
        else:
            break

    if active_rebal is None:
        continue

    active_holdings = holdings_history[active_rebal]['holdings']
    active_cash = holdings_history[active_rebal]['cash']

    # Precios de cierre de ese día
    prices_day = (parquet[parquet['date'] == day]
                  .set_index('symbol')['close'])

    # Valor total = cash + valor de posiciones
    total_value = active_cash
    for ticker, shares in active_holdings.items():
        if ticker in prices_day.index:
            total_value += shares * prices_day[ticker]

    portfolio_values.append({'date': day, 'portfolio_value': total_value})

portfolio_daily = pd.DataFrame(portfolio_values)
portfolio_daily['return_pct'] = (portfolio_daily['portfolio_value'] / INITIAL_CAPITAL - 1) * 100

print(f"Días calculados: {len(portfolio_daily)}")
print(portfolio_daily.head())
print(portfolio_daily.tail())

Días calculados: 2767
        date  portfolio_value  return_pct
0 2015-01-30    249454.953125   -0.218016
1 2015-02-02    249930.906250   -0.027639
2 2015-02-03    251893.375000    0.757349
3 2015-02-04    252049.312500    0.819731
4 2015-02-05    254808.000000    1.923203
           date  portfolio_value  return_pct
2762 2026-01-26      1434784.375  473.913757
2763 2026-01-27      1480472.875  492.189178
2764 2026-01-28      1523071.000  509.228424
2765 2026-01-29      1514669.000  505.867584
2766 2026-01-30      1447109.375  478.843750


In [11]:
# ------------------------------------------------------------
# 6. GUARDAMOS OUTPUTS PARA EL NOTEBOOK 5
# ------------------------------------------------------------

portfolio_daily.to_parquet("resources/portfolio_daily.parquet", index=False)
print("✅ portfolio_daily.parquet guardado")

# Resumen de comisiones
print(f"\nResumen del backtesting:")
print(f"Capital inicial:      ${INITIAL_CAPITAL:,.2f}")
print(f"Capital final:        ${portfolio_daily['portfolio_value'].iloc[-1]:,.2f}")
print(f"Retorno acumulado:    {portfolio_daily['return_pct'].iloc[-1]:.2f}%")
print(f"Comisiones totales:   ${total_commissions:,.2f}")
print(f"% del capital inicial: {total_commissions/INITIAL_CAPITAL*100:.2f}%")

✅ portfolio_daily.parquet guardado

Resumen del backtesting:
Capital inicial:      $250,000.00
Capital final:        $1,447,109.38
Retorno acumulado:    478.84%
Comisiones totales:   $106,769.50
% del capital inicial: 42.71%
